In [1]:
pip install pgmpy


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pgmpy.models import MarkovNetwork
from pgmpy.factors.discrete import DiscreteFactor

# Create the model and add nodes
model = MarkovNetwork()
model.add_nodes_from(['A', 'B', 'C', 'D', 'E'])

# edges
model.add_edges_from([('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('E', 'A')])


c:\Users\micha\anaconda3\envs\medical_bayes\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
phi_A_B = DiscreteFactor(['A', 'B'], [2, 2], [1, 50, 50, 1])
phi_B_C = DiscreteFactor(['B', 'C'], [2, 2], [1, 100, 100, 1])
phi_C_D = DiscreteFactor(['C', 'D'], [2, 2], [1, 30, 30, 1])
phi_D_E = DiscreteFactor(['D', 'E'], [2, 2], [1, 20, 20, 1])
phi_E_A = DiscreteFactor(['E', 'A'], [2, 2], [1, 10, 10, 1])
print(phi_A_B,phi_B_C,phi_C_D,phi_D_E,phi_E_A)

# Add factors to model
model.add_factors(phi_A_B, phi_B_C, phi_C_D, phi_D_E, phi_E_A)
print(model)


MarkovNetwork with 5 nodes and 5 edges


In [4]:
chordal_graph = model.triangulate()
print("Edges of triangulated chordal graph:")
print(chordal_graph.edges())


Edges of triangulated chordal graph:
[('A', 'B'), ('A', 'E'), ('A', 'D'), ('B', 'C'), ('B', 'D'), ('E', 'D'), ('C', 'D')]


In [5]:
from pgmpy.inference import BeliefPropagation

bp = BeliefPropagation(model)
marginal_C = bp.query(variables=['C'])
print("Marginal Distribution of C:")
print(marginal_C)


Marginal Distribution of C:
+------+----------+
| C    |   phi(C) |
+======+==========+
| C(0) |   0.5000 |
+------+----------+
| C(1) |   0.5000 |
+------+----------+


In [6]:
map_assignment = bp.map_query(variables=['A', 'B', 'C', 'D', 'E'])
print("MAP Assignment:")
print(map_assignment)


MAP Assignment:
{'A': 0, 'B': 1, 'C': 0, 'D': 1, 'E': 0}


In [7]:
# Step 1: Remove old factor (optional)
# Recreate the model to avoid duplicates
model = MarkovNetwork()
model.add_nodes_from(['A', 'B', 'C', 'D', 'E'])
model.add_edges_from([('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('E', 'A')])

# Step 2: Redefine the factors with modified phi_C_D
phi_A_B = DiscreteFactor(['A', 'B'], [2, 2], [1, 50, 50, 1])
phi_B_C = DiscreteFactor(['B', 'C'], [2, 2], [1, 100, 100, 1])
phi_C_D_new = DiscreteFactor(['C', 'D'], [2, 2], [100, 1, 1, 100])  # changed
phi_D_E = DiscreteFactor(['D', 'E'], [2, 2], [1, 20, 20, 1])
phi_E_A = DiscreteFactor(['E', 'A'], [2, 2], [1, 10, 10, 1])

# Step 3: Add updated factors
model.add_factors(phi_A_B, phi_B_C, phi_C_D_new, phi_D_E, phi_E_A)

# Step 4: Inference
bp = BeliefPropagation(model)

# Re-run marginal inference
marginal_C_new = bp.query(variables=['C'])
print("New Marginal Distribution of C after changing phi_C_D:")
print(marginal_C_new)

# Re-run MAP inference
map_assignment_new = bp.map_query(variables=['A', 'B', 'C', 'D', 'E'])
print("New MAP Assignment after changing phi_C_D:")
print(map_assignment_new)


New Marginal Distribution of C after changing phi_C_D:
+------+----------+
| C    |   phi(C) |
+======+==========+
| C(0) |   0.5000 |
+------+----------+
| C(1) |   0.5000 |
+------+----------+
New MAP Assignment after changing phi_C_D:
{'A': 0, 'B': 1, 'C': 0, 'D': 0, 'E': 1}
